In [1]:
from data_transformer import DataTransformer
from data_sampler import DataSampler
from ArchCTGAN import CTGAN_P
from SNIP_CTGAN import SNIP_ctgan
from prepare_data import preprocess_data_czech
import pandas as pd
import numpy as np

2023-06-21 13:29:01.195327: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [4]:
df = pd.read_csv('../CTGAN/tr_by_acct_w_age.csv')
raw_data = preprocess_data_czech(df)

In [5]:
final_raw = raw_data[['amount', 'tcode','td']]

In [3]:
transformer = DataTransformer()
transformer.fit(final_raw, discrete_columns=('tcode'))
data_t = transformer.transform(final_raw)      #matrix of transformed data





NameError: name 'final_raw' is not defined

In [5]:
import pickle

# Save an object
with open('object.pkl', 'wb') as file:
    pickle.dump(transformer, file)

# Save a matrix
with open('matrix.pkl', 'wb') as file:
    pickle.dump(data_t, file)

In [6]:
import pickle
# Load an object
with open('object.pkl', 'rb') as file:
    transformer = pickle.load(file)

# Load a matrix
with open('matrix.pkl', 'rb') as file:
    data_t = pickle.load(file)

In [7]:
output_info = transformer.output_info_list

In [8]:
account_id_counts = raw_data['account_id'].value_counts().sort_index()
trans_sizes = np.array(account_id_counts)
assert sum(trans_sizes) == data_t.shape[0]
transactions = np.split(data_t, np.cumsum(trans_sizes)[:-1])    

In [9]:
log_frequency = True
#sampler = DataSampler(data_t, output_info, log_frequency)
sampler = DataSampler(data_t,transactions,output_info, log_frequency)

In [10]:
model = CTGAN_P(sampler, transformer)

In [11]:
batchsize = 50
target_sparsity = 0.1
final_w_disc, final_mask_disc, final_w_gen, final_mask_gen = SNIP_ctgan(model, batchsize, target_sparsity)

2023-06-21 13:31:14.575011: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-06-21 13:31:14.576040: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-06-21 13:31:14.603830: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: Quadro RTX 8000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 47.45GiB deviceMemoryBandwidth: 625.94GiB/s
2023-06-21 13:31:14.604037: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:af:00.0 name: Quadro RTX 8000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 47.45GiB deviceMemoryBandwidth: 625.94GiB/s
2023-06-21 13:31:14.604065: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-06-21 13:3

In [12]:
#build_generator
generator = model.make_layers_gen(final_w_gen, final_mask_gen)
#build discriminator
discriminator = model.make_layers_disc(final_w_disc, final_mask_disc)

In [13]:
from train import Train
train = Train(transformer, sampler, generator, discriminator,  epochs=10)

In [15]:
train.train(raw_data)

Epoch 1, Loss G: -1.8632, Loss D: -0.0670


KeyboardInterrupt: 

In [14]:
data = train.synthesise_data3(800, raw_data)
data.to_csv('synth.csv', index=False)

In [16]:
s = pd.read_csv('synth.csv')
s

,amount,tcode,td,account_id,cumulative_td,datetime
0,205.894742,CREDIT__CREDIT IN CASH__nan,14,1,14,1994-05-14
1,18863.054559,CREDIT__COLLECTION FROM ANOTHER BANK__OLD AGE ...,10,1,24,1994-05-24
2,21302.055024,DEBIT__CASH WITHDRAWAL__INSURANCE PAYMENT,18,1,42,1994-06-11
3,25642.723585,DEBIT__CASH WITHDRAWAL__SANCTION INTEREST,5,1,47,1994-06-16
4,19214.150144,DEBIT__REMITTANCE TO ANOTHER BANK__LOAN PAYMENT,3,1,50,1994-06-19
...,...,...,...,...,...,...
185066,46649.638478,CREDIT__COLLECTION FROM ANOTHER BANK__OLD AGE ...,18,800,1714,2003-08-10
185067,145.552530,DEBIT__CASH WITHDRAWAL__,11,800,1725,2003-08-21
185068,10245.577352,DEBIT__CASH WITHDRAWAL__SANCTION INTEREST,1,800,1726,2003-08-22
185069,43844.689979,DEBIT__CREDIT CARD WITHDRAWAL__nan,4,800,1730,2003-08-26
